In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_text_label.csv


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_amzn = pd.read_csv('amzn_title_processedText_direction_date.csv')


In [ ]:
df_amzn.head()

,title,text,direction,text_without_stopwords,processedText,prediction_date
0,6 things to know before the opening bell,click chart for more in-depth data. 1. big tec...,-1,click chart in-depth data 1. big tech senate b...,click chart in-depth data 1. big tech senate b...,2018-09-05
1,"Netflix And The Competition - Netflix, Inc. (N...",investment thesis netflix ( nflx ) is a remark...,-1,investment thesis netflix nflx remarkable serv...,investment thesis netflix nflx remarkable serv...,2018-09-05
2,Pandora Stock Gets a Boost — Buy Now,mannkind surges more than 50% >>> read more\nb...,-1,mannkind surges 50 read breaking news sponsore...,surges 50 read breaking news sponsored home st...,2018-09-05
3,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,"by mark decambre, marketwatch , ryan vlastelic...",-1,mark decambre marketwatch ryan vlastelica tech...,mark decambre marketwatch ryan vlastelica tech...,2018-09-05
4,"New Month, Same Old Story: Trade Fears Persist...","it may be a new month, but the same trade conc...",-1,may new month trade concerns haunted market ye...,may new month trade concerns market year remai...,2018-09-05


In [ ]:
!pip install scikeras
from sklearn.model_selection import TimeSeriesSplit, train_test_split, KFold, cross_validate, cross_val_score, cross_val_predict
from scikeras.wrappers import KerasClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def split_generator(df):
  cv=[]
  df['prediction_date'] = pd.to_datetime(df['prediction_date'], errors='coerce')
  groups = df.groupby(df['prediction_date'].dt.to_period("M")).groups
  split_groups = [value for (key, value) in groups.items()] 

  print('spilt number:' + str(len(split_groups)))
  for i in range(len(split_groups)):
    data_index=[]
    for j in range(len(split_groups)):
      if i != j:
        data_index.extend(split_groups[j].tolist())
    cv.append((data_index, split_groups[i].tolist()))
    # yield data_index, split_groups[i].tolist()
  
  return cv

In [ ]:
amzn_cv=split_generator(df_amzn)


spilt number:15


In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import Adam, SGD

In [ ]:
def lstm_model(shape):
  layers = [LSTM(128,input_shape=(1,shape),dropout=0.2), Dense(2)]
  model = tf.keras.models.Sequential(layers)
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(),metrics=['accuracy'])
  keras_clf = KerasClassifier(model = model, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=tf.keras.optimizers.SGD(), epochs=40, batch_size=64, verbose=2,validation_split=0.2)

  
  return keras_clf

In [ ]:
def lstm_cv(stock, desc, features, label, cv, print_summary=False, print_box_plot=False):
    print('Running: ', len(features))
    X = features
    y = label
    model = lstm_model(X.shape[1])
    print(X.shape)
    TrainX = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    
    print("model: ", model)

    results = pd.DataFrame(
        cross_validate(model, TrainX, y, cv=cv, scoring=['accuracy', 'precision', 'recall', 'f1']))

    if print_box_plot:
        results.drop('fit_time', axis=1).plot.box(figsize=(10, 5))

    summary = results.describe()
    if print_summary:
        print(summary.loc['mean', :])

    summary = summary.loc['mean', :]
    summary['stock'] = stock
    summary['desc'] = desc
    #print(summary.shape)
    return summary

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


In [ ]:
#title_amzn=df_amzn['title'].tolist()
text_amzn=df_amzn['processedText'].tolist()
label_amzn=df_amzn['direction'].to_numpy()


#print(title_amzn[0])
print(text_amzn[0])
print(label_amzn[0])

click chart in-depth data 1. big tech senate big tech executives coming capitol hill wednesday testify election meddling facebook fb coo sheryl sandberg jack dorsey ceo twitter twtr appear senate intelligence committee discuss actions companies taken foreign influence campaigns targeting 2018 midterm elections committee requested high-profile executive google owns youtube also appear testify google googl parent company alphabet however declined send one top people 2. emerging markets fears emerging markets currencies tumbled dollar wednesday mexican south indian among biggest losers rising us interest rates political global trade war recent months exposed multiple emerging markets turkey argentina hardest hit investors worried adding fears markit index measures emerging markets manufacturing fell fresh low august watch cnnmoney now today p.m. et 3. global stocks overview us stock futures pointing lower european markets opened down exchanges london frankfurt paris dropping roughly 0.5 a

In [ ]:
class_label_amzn = (label_amzn + 1) / 2

In [ ]:
text_tokenized_amzn, text_tokenizer_amzn=tokenize(text_amzn)
text_padded_amzn=pad_sequences(text_tokenized_amzn, padding='post')

In [ ]:
print(text_padded_amzn[0])

[480 247 481 ...   0   0   0]


In [ ]:
summary_amzn = lstm_cv('AMZN', 'AMZN using news body', text_padded_amzn, class_label_amzn, amzn_cv)

Running:  19078
(19078, 12487)
model:  KerasClassifier(
	model=<keras.engine.sequential.Sequential object at 0x7f65a5c06e10>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<keras.optimizers.optimizer_v2.gradient_descent.SGD object at 0x7f65a5be7390>
	loss=<keras.losses.SparseCategoricalCrossentropy object at 0x7f65a5c2e7d0>
	metrics=['accuracy']
	batch_size=64
	validation_batch_size=None
	verbose=2
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=40
	class_weight=None
)


Epoch 1/40
239/239 - 21s - loss: 0.7075 - accuracy: 0.2641 - val_loss: 0.7098 - val_accuracy: 0.2308 - 21s/epoch - 90ms/step
Epoch 2/40
239/239 - 20s - loss: 0.6949 - accuracy: 0.2536 - val_loss: 0.7008 - val_accuracy: 0.2206 - 20s/epoch - 83ms/step
Epoch 3/40
239/239 - 18s - loss: 0.6965 - accuracy: 0.2701 - val_loss: 0.6917 - val_accuracy: 0.8117 - 18s/epoch - 77ms/step
Epoch 4/40
239/239 - 20s - loss: 0.6937 - accuracy: 0.2770 - val_loss: 0.6972 - val_accuracy: 0.3617 - 20s/epoch - 84ms/step
Epoch 5/40
239/239 - 17s - loss: 0.6932 - accuracy: 0.2540 - val_loss: 0.7114 - val_accuracy: 0.0614 - 17s/epoch - 70ms/step
Epoch 6/40
239/239 - 17s - loss: 0.6935 - accuracy: 0.2327 - val_loss: 0.7013 - val_accuracy: 0.2206 - 17s/epoch - 70ms/step
Epoch 7/40
239/239 - 19s - loss: 0.6929 - accuracy: 0.2594 - val_loss: 0.7077 - val_accuracy: 0.0732 - 19s/epoch - 81ms/step
Epoch 8/40
239/239 - 17s - loss: 0.6918 - accuracy: 0.1899 - val_loss: 0.7010 - val_accuracy: 0.1770 - 17s/epoch - 73ms/step


Epoch 1/40
219/219 - 18s - loss: 0.7112 - accuracy: 0.5129 - val_loss: 0.6959 - val_accuracy: 0.5559 - 18s/epoch - 83ms/step
Epoch 2/40
219/219 - 15s - loss: 0.6990 - accuracy: 0.5496 - val_loss: 0.6930 - val_accuracy: 0.7108 - 15s/epoch - 67ms/step
Epoch 3/40
219/219 - 17s - loss: 0.6960 - accuracy: 0.5326 - val_loss: 0.6947 - val_accuracy: 0.6205 - 17s/epoch - 76ms/step
Epoch 4/40
219/219 - 15s - loss: 0.6945 - accuracy: 0.5299 - val_loss: 0.6990 - val_accuracy: 0.0886 - 15s/epoch - 68ms/step
Epoch 5/40
219/219 - 15s - loss: 0.6938 - accuracy: 0.4780 - val_loss: 0.6952 - val_accuracy: 0.3710 - 15s/epoch - 67ms/step
Epoch 6/40
219/219 - 17s - loss: 0.6939 - accuracy: 0.4578 - val_loss: 0.6932 - val_accuracy: 0.6828 - 17s/epoch - 77ms/step
Epoch 7/40
219/219 - 15s - loss: 0.6936 - accuracy: 0.4765 - val_loss: 0.6949 - val_accuracy: 0.3858 - 15s/epoch - 68ms/step
Epoch 8/40
219/219 - 15s - loss: 0.6945 - accuracy: 0.5040 - val_loss: 0.6958 - val_accuracy: 0.5967 - 15s/epoch - 70ms/step


Epoch 1/40
224/224 - 20s - loss: 0.7070 - accuracy: 0.3455 - val_loss: 0.7062 - val_accuracy: 0.1839 - 20s/epoch - 87ms/step
Epoch 2/40
224/224 - 15s - loss: 0.6954 - accuracy: 0.2921 - val_loss: 0.6997 - val_accuracy: 0.4488 - 15s/epoch - 68ms/step
Epoch 3/40
224/224 - 17s - loss: 0.6942 - accuracy: 0.2886 - val_loss: 0.6994 - val_accuracy: 0.2632 - 17s/epoch - 74ms/step
Epoch 4/40
224/224 - 15s - loss: 0.6940 - accuracy: 0.3030 - val_loss: 0.7116 - val_accuracy: 0.2094 - 15s/epoch - 68ms/step
Epoch 5/40
224/224 - 16s - loss: 0.6943 - accuracy: 0.1700 - val_loss: 0.7030 - val_accuracy: 0.1484 - 16s/epoch - 71ms/step
Epoch 6/40
224/224 - 16s - loss: 0.6929 - accuracy: 0.1958 - val_loss: 0.7050 - val_accuracy: 0.0529 - 16s/epoch - 74ms/step
Epoch 7/40
224/224 - 15s - loss: 0.6927 - accuracy: 0.1356 - val_loss: 0.6959 - val_accuracy: 0.3810 - 15s/epoch - 68ms/step
Epoch 8/40
224/224 - 17s - loss: 0.6919 - accuracy: 0.1830 - val_loss: 0.6992 - val_accuracy: 0.1982 - 17s/epoch - 74ms/step


Epoch 1/40
223/223 - 20s - loss: 0.7062 - accuracy: 0.3060 - val_loss: 0.6982 - val_accuracy: 0.3732 - 20s/epoch - 89ms/step
Epoch 2/40
223/223 - 18s - loss: 0.6943 - accuracy: 0.4110 - val_loss: 0.7001 - val_accuracy: 0.3068 - 18s/epoch - 79ms/step
Epoch 3/40
223/223 - 16s - loss: 0.6948 - accuracy: 0.2231 - val_loss: 0.7047 - val_accuracy: 0.3614 - 16s/epoch - 70ms/step
Epoch 4/40
223/223 - 16s - loss: 0.6928 - accuracy: 0.2088 - val_loss: 0.6978 - val_accuracy: 0.1257 - 16s/epoch - 70ms/step
Epoch 5/40
223/223 - 18s - loss: 0.6918 - accuracy: 0.1940 - val_loss: 0.6964 - val_accuracy: 0.2708 - 18s/epoch - 80ms/step
Epoch 6/40
223/223 - 16s - loss: 0.6914 - accuracy: 0.1549 - val_loss: 0.7012 - val_accuracy: 0.0413 - 16s/epoch - 70ms/step
Epoch 7/40
223/223 - 16s - loss: 0.6925 - accuracy: 0.2525 - val_loss: 0.6980 - val_accuracy: 0.2545 - 16s/epoch - 70ms/step
Epoch 8/40
223/223 - 17s - loss: 0.6914 - accuracy: 0.1489 - val_loss: 0.7002 - val_accuracy: 0.0661 - 17s/epoch - 78ms/step


...

Epoch 1/40
217/217 - 19s - loss: 0.7124 - accuracy: 0.3663 - val_loss: 0.7331 - val_accuracy: 0.2123 - 19s/epoch - 88ms/step
Epoch 2/40
217/217 - 15s - loss: 0.6989 - accuracy: 0.3332 - val_loss: 0.6939 - val_accuracy: 0.6981 - 15s/epoch - 69ms/step
Epoch 3/40
217/217 - 17s - loss: 0.6942 - accuracy: 0.2394 - val_loss: 0.7156 - val_accuracy: 0.0188 - 17s/epoch - 79ms/step
Epoch 4/40
217/217 - 15s - loss: 0.6934 - accuracy: 0.2157 - val_loss: 0.7107 - val_accuracy: 0.0784 - 15s/epoch - 70ms/step
Epoch 5/40
217/217 - 17s - loss: 0.6950 - accuracy: 0.2374 - val_loss: 0.7041 - val_accuracy: 0.1356 - 17s/epoch - 79ms/step
Epoch 6/40
217/217 - 17s - loss: 0.6934 - accuracy: 0.1789 - val_loss: 0.6956 - val_accuracy: 0.6504 - 17s/epoch - 79ms/step
Epoch 7/40
217/217 - 16s - loss: 0.6929 - accuracy: 0.2161 - val_loss: 0.7033 - val_accuracy: 0.3141 - 16s/epoch - 72ms/step
Epoch 8/40
217/217 - 17s - loss: 0.6920 - accuracy: 0.2267 - val_loss: 0.7062 - val_accuracy: 0.0564 - 17s/epoch - 80ms/step


Epoch 1/40
220/220 - 21s - loss: 0.7088 - accuracy: 0.4130 - val_loss: 0.7095 - val_accuracy: 0.0757 - 21s/epoch - 96ms/step
Epoch 2/40
220/220 - 15s - loss: 0.6969 - accuracy: 0.2884 - val_loss: 0.6951 - val_accuracy: 0.5727 - 15s/epoch - 69ms/step
Epoch 3/40
220/220 - 15s - loss: 0.6943 - accuracy: 0.2474 - val_loss: 0.7015 - val_accuracy: 0.1169 - 15s/epoch - 68ms/step
Epoch 4/40
220/220 - 18s - loss: 0.6940 - accuracy: 0.2102 - val_loss: 0.6970 - val_accuracy: 0.2620 - 18s/epoch - 80ms/step
Epoch 5/40
220/220 - 16s - loss: 0.6934 - accuracy: 0.2017 - val_loss: 0.7009 - val_accuracy: 0.0336 - 16s/epoch - 73ms/step
Epoch 6/40
220/220 - 18s - loss: 0.6924 - accuracy: 0.1428 - val_loss: 0.7031 - val_accuracy: 0.1297 - 18s/epoch - 80ms/step
Epoch 7/40
220/220 - 16s - loss: 0.6923 - accuracy: 0.1716 - val_loss: 0.7025 - val_accuracy: 0.0108 - 16s/epoch - 71ms/step
Epoch 8/40
220/220 - 15s - loss: 0.6938 - accuracy: 0.1215 - val_loss: 0.6996 - val_accuracy: 0.3055 - 15s/epoch - 70ms/step


Epoch 1/40
219/219 - 19s - loss: 0.7071 - accuracy: 0.3248 - val_loss: 0.7081 - val_accuracy: 0.2374 - 19s/epoch - 88ms/step
Epoch 2/40
219/219 - 17s - loss: 0.6985 - accuracy: 0.3419 - val_loss: 0.7115 - val_accuracy: 0.1373 - 17s/epoch - 79ms/step
Epoch 3/40
219/219 - 15s - loss: 0.6968 - accuracy: 0.2880 - val_loss: 0.7108 - val_accuracy: 0.0317 - 15s/epoch - 70ms/step
Epoch 4/40
219/219 - 15s - loss: 0.6949 - accuracy: 0.2538 - val_loss: 0.7034 - val_accuracy: 0.2334 - 15s/epoch - 70ms/step
Epoch 5/40
219/219 - 17s - loss: 0.6934 - accuracy: 0.1732 - val_loss: 0.6991 - val_accuracy: 0.1455 - 17s/epoch - 79ms/step
Epoch 6/40
219/219 - 15s - loss: 0.6923 - accuracy: 0.1755 - val_loss: 0.7040 - val_accuracy: 0.1535 - 15s/epoch - 69ms/step
Epoch 7/40
219/219 - 15s - loss: 0.6933 - accuracy: 0.2047 - val_loss: 0.7044 - val_accuracy: 0.0848 - 15s/epoch - 70ms/step
Epoch 8/40
219/219 - 17s - loss: 0.6915 - accuracy: 0.2093 - val_loss: 0.7009 - val_accuracy: 0.3938 - 17s/epoch - 80ms/step


Epoch 1/40
238/238 - 21s - loss: 0.7034 - accuracy: 0.3570 - val_loss: 0.7079 - val_accuracy: 0.1622 - 21s/epoch - 89ms/step
Epoch 2/40
238/238 - 18s - loss: 0.6962 - accuracy: 0.2867 - val_loss: 0.7023 - val_accuracy: 0.1861 - 18s/epoch - 77ms/step
Epoch 3/40
238/238 - 17s - loss: 0.6942 - accuracy: 0.2440 - val_loss: 0.6991 - val_accuracy: 0.1482 - 17s/epoch - 71ms/step
Epoch 4/40
238/238 - 19s - loss: 0.6929 - accuracy: 0.2241 - val_loss: 0.7085 - val_accuracy: 0.3482 - 19s/epoch - 81ms/step
Epoch 5/40
238/238 - 17s - loss: 0.6934 - accuracy: 0.2928 - val_loss: 0.7039 - val_accuracy: 0.0909 - 17s/epoch - 70ms/step
Epoch 6/40
238/238 - 19s - loss: 0.6926 - accuracy: 0.2498 - val_loss: 0.7111 - val_accuracy: 0.0539 - 19s/epoch - 80ms/step
Epoch 7/40
238/238 - 17s - loss: 0.6933 - accuracy: 0.2136 - val_loss: 0.7109 - val_accuracy: 0.0578 - 17s/epoch - 70ms/step
Epoch 8/40
238/238 - 17s - loss: 0.6930 - accuracy: 0.2249 - val_loss: 0.7092 - val_accuracy: 0.0489 - 17s/epoch - 70ms/step


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
summary_amzn

fit_time                    677.487686
score_time                     1.45617
test_accuracy                 0.438907
test_precision                0.450804
test_recall                     0.7292
test_f1                       0.539529
stock                             AMZN
desc              AMZN using news body
Name: mean, dtype: object

In [ ]:
!pwd

/content/drive/Shareddrives/CSE573/preprocessed


In [ ]:
summary_amzn.to_csv('text_kfold_amzn.csv')